# Load and Preprocess

In [1340]:
import pandas as pd
# Remove the limit on the number of columns displayed
pd.set_option('display.max_columns', None)

In [1341]:
table_df = pd.read_csv("finaltable1.csv")

In [1342]:
table_df.tail()

,Unnamed: 0.1,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,Score,xG.1,Attendance,Venue,Referee,Away Win Odds,Home Win Odds,Draw Odds,StadiumID,Latitude,Longitude,Datetime,Temperature,Precipitation,H2H_Home_Wins_Last_4,H2H_Away_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A
364,364,497785,Newcastle Utd,Everton,NaN,NaN,NaN,15,5,5,5,19,21,-2,20,1.0,14,3,5,6,14,21,-7,14,1.0,2025-05-25,15:00,38,Sun,NaN,NaN,NaN,NaN,St James' Park,NaN,NaN,NaN,NaN,5.0,54.9756,-1.6217,2025-05-25T15:00:00,NaN,NaN,0,0,1,2025-05-25_Newcastle Utd_Everton,EVE,NEW,NaN,NaN,False,False,https://understat.com/match/26973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,375,38.0,2,4,Newcastle Utd,Everton,2025-05-25_Newcastle Utd_Everton,NaN,NaN,NaN,NaN,NaN
365,365,497786,Nottingham Forest,Chelsea,NaN,NaN,NaN,15,7,4,4,19,18,1,25,1.2,15,9,4,2,35,18,17,31,2.6,2025-05-25,15:00,38,Sun,NaN,NaN,NaN,NaN,The City Ground,NaN,NaN,NaN,NaN,6.0,52.9399,-1.1325,2025-05-25T15:00:00,NaN,NaN,0,0,1,2025-05-25_Nottingham Forest_Chelsea,CHE,NOT,NaN,NaN,False,False,https://understat.com/match/26972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,376,38.0,4,2,Nottingham Forest,Chelsea,2025-05-25_Nottingham Forest_Chelsea,NaN,NaN,NaN,NaN,NaN
366,366,497787,Southampton,Arsenal,NaN,NaN,NaN,15,1,2,12,11,31,-20,5,0.2,15,8,5,2,29,15,14,29,2.2,2025-05-25,15:00,38,Sun,NaN,NaN,NaN,NaN,St Mary's Stadium,NaN,NaN,NaN,NaN,15.0,50.9058,-1.3913,2025-05-25T15:00:00,NaN,NaN,0,1,0,2025-05-25_Southampton_Arsenal,ARS,SOU,NaN,NaN,False,False,https://understat.com/match/26977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,377,38.0,5,2,Southampton,Arsenal,2025-05-25_Southampton_Arsenal,NaN,NaN,NaN,NaN,NaN
367,367,497788,Tottenham,Brighton,NaN,NaN,NaN,15,6,2,7,31,19,12,20,0.8,15,6,6,3,25,22,3,24,1.6,2025-05-25,15:00,38,Sun,NaN,NaN,NaN,NaN,Tottenham Hotspur Stadium,NaN,NaN,NaN,NaN,16.0,51.6047,-0.0664,2025-05-25T15:00:00,NaN,NaN,0,1,0,2025-05-25_Tottenham_Brighton,BRI,TOT,NaN,NaN,False,False,https://understat.com/match/26976,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,378,38.0,2,4,Tottenham,Brighton,2025-05-25_Tottenham_Brighton,NaN,NaN,NaN,NaN,NaN
368,368,497789,Wolves,Brentford,NaN,NaN,NaN,15,2,3,10,23,38,-15,9,1.2,15,7,2,6,31,28,3,23,2.0,2025-05-25,15:00,38,Sun,NaN,NaN,NaN,NaN,Molineux Stadium,NaN,NaN,NaN,NaN,19.0,52.5903,-2.1303,2025-05-25T15:00:00,NaN,NaN,0,1,0,2025-05-25_Wolves_Brentford,BRE,WOL,NaN,NaN,False,False,https://understat.com/match/26978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,379,38.0,2,2,Wolves,Brentford,2025-05-25_Wolves_Brentford,NaN,NaN,NaN,NaN,NaN


In [1343]:
len(table_df)

369

# Data Preprocessing

## Handling Missing Values

In [1346]:
def check_missing_values():
    # Check for missing values in the dataset
    missing_values = table_df.isnull().sum()
    
    # Filter for columns with missing values
    missing_values = missing_values[missing_values > 0]
    
    # Display missing values
    print("Columns with Missing Values:")
    print(missing_values)


In [1347]:
check_missing_values()

Columns with Missing Values:
Home Score               220
Away Score               220
Winner                   220
xG                       220
Score                    220
xG.1                     220
Attendance               220
Referee                  220
Away Win Odds            360
Home Win Odds            360
Draw Odds                360
Temperature              239
Precipitation            239
away_team_code            24
home_team_code            24
home_xg                  220
away_xg                  220
is_result                 24
has_data                  24
url                       24
away_points              220
away_expected_points     220
away_np_xg               220
away_np_xg_difference    220
away_ppda                220
away_deep_completions    220
home_points              220
home_expected_points     220
home_np_xg               220
home_np_xg_difference    220
home_ppda                220
home_deep_completions    220
HomeTeam                 220
AwayTeam      

Store null fixtures for using to prediction later

In [1349]:
# Create a new DataFrame with rows where 'Home Score' or 'Away Score' is null
null_scores_df = table_df[table_df['Home Score'].isnull() | table_df['Away Score'].isnull()]


In [1350]:
null_scores_df.head()

,Unnamed: 0.1,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,Score,xG.1,Attendance,Venue,Referee,Away Win Odds,Home Win Odds,Draw Odds,StadiumID,Latitude,Longitude,Datetime,Temperature,Precipitation,H2H_Home_Wins_Last_4,H2H_Away_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A
149,149,497561,Arsenal,Everton,NaN,NaN,NaN,15,8,5,2,29,15,14,29,2.2,14,3,5,6,14,21,-7,14,1.0,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,NaN,Emirates Stadium,NaN,13.0,1.20,5.50,3.0,51.5550,-0.1086,2024-12-14T15:00:00,NaN,NaN,0,0,0,2024-12-14_Arsenal_Everton,EVE,ARS,NaN,NaN,False,False,https://understat.com/match/26755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,150,16.0,2,5,Arsenal,Everton,2024-12-14_Arsenal_Everton,NaN,NaN,NaN,NaN,NaN
150,150,497564,Liverpool,Fulham,NaN,NaN,NaN,14,11,2,1,29,11,18,35,2.6,15,6,5,4,22,20,2,23,1.6,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,NaN,Anfield,NaN,9.0,1.29,5.50,20.0,53.4308,-2.9608,2024-12-14T15:00:00,NaN,NaN,0,0,0,2024-12-14_Liverpool_Fulham,FLH,LIV,NaN,NaN,False,False,https://understat.com/match/26752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151,16.0,2,5,Liverpool,Fulham,2024-12-14_Liverpool_Fulham,NaN,NaN,NaN,NaN,NaN
151,151,497566,Newcastle Utd,Leicester City,NaN,NaN,NaN,15,5,5,5,19,21,-2,20,1.0,15,3,5,7,21,30,-9,14,0.8,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,NaN,St James' Park,NaN,NaN,NaN,NaN,5.0,54.9756,-1.6217,2024-12-14T15:00:00,NaN,NaN,0,0,0,2024-12-14_Newcastle Utd_Leicester City,LEI,NEW,NaN,NaN,False,False,https://understat.com/match/26760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,152,16.0,2,4,Newcastle Utd,Leicester City,2024-12-14_Newcastle Utd_Leicester City,NaN,NaN,NaN,NaN,NaN
152,152,497569,Wolves,Ipswich Town,NaN,NaN,NaN,15,2,3,10,23,38,-15,9,1.2,15,1,6,8,14,27,-13,9,0.8,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,NaN,Molineux Stadium,NaN,NaN,NaN,NaN,19.0,52.5903,-2.1303,2024-12-14T15:00:00,NaN,NaN,0,0,0,2024-12-14_Wolves_Ipswich Town,IPS,WOL,NaN,NaN,False,False,https://understat.com/match/26758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,153,16.0,2,2,Wolves,Ipswich Town,2024-12-14_Wolves_Ipswich Town,NaN,NaN,NaN,NaN,NaN
153,153,497567,Nottingham Forest,Aston Villa,NaN,NaN,NaN,15,7,4,4,19,18,1,25,1.2,15,7,4,4,23,23,0,25,1.4,2024-12-14,17:30,16,Sat,NaN,NaN,NaN,NaN,The City Ground,NaN,2.5,2.40,2.75,6.0,52.9399,-1.1325,2024-12-14T17:30:00,NaN,NaN,0,0,0,2024-12-14_Nottingham Forest_Aston Villa,AVL,NOT,NaN,NaN,False,False,https://understat.com/match/26761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,154,16.0,3,2,Nottingham Forest,Aston Villa,2024-12-14_Nottingham Forest_Aston Villa,NaN,NaN,NaN,NaN,NaN


Remove matches that have not been played..

In [1352]:
table_df = table_df.dropna(subset=['Home Score', 'Away Score'])
# Reset the index after filtering
table_df.reset_index(drop=True, inplace=True)

In [1353]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    149
Home Win Odds    149
Draw Odds        149
Temperature       19
Precipitation     19
dtype: int64


In [1354]:
# Fill missing 'xG' values with corresponding 'home_xg' values for the same row
table_df['xG'] = table_df.apply(
    lambda row: row['home_xg'] if pd.isnull(row['xG']) else row['xG'],
    axis=1
)
table_df['xG.1'] = table_df.apply(
    lambda row: row['away_xg'] if pd.isnull(row['xG.1']) else row['xG.1'],
    axis=1
)

# Verify that there are no more missing values in 'xG'
missing_xg_after = table_df['xG'].isnull().sum()
print(f"Remaining missing values in 'xG': {missing_xg_after}")
missing_xg_after = table_df['xG.1'].isnull().sum()
print(f"Remaining missing values in 'xG.1': {missing_xg_after}")


Remaining missing values in 'xG': 0
Remaining missing values in 'xG.1': 0


In [1355]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    149
Home Win Odds    149
Draw Odds        149
Temperature       19
Precipitation     19
dtype: int64


Drop the score column because we already have home scores and away scores

In [1357]:
# Drop the 'Score' column from the dataset
table_df.drop(columns=['Score','Precipitation'], inplace=True)

In [1358]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    149
Home Win Odds    149
Draw Odds        149
Temperature       19
dtype: int64


In [1359]:
# Fill missing 'Attendance' values with the median attendance
median_attendance = table_df['Attendance'].median()
table_df['Attendance'].fillna(median_attendance, inplace=True)

# Verify there are no missing values in 'Attendance'
print(f"Remaining missing values in 'Attendance': {table_df['Attendance'].isnull().sum()}")


Remaining missing values in 'Attendance': 0


In [1360]:
# Filter rows where 'Referee' is null
rows_with_missing_referee = table_df[table_df['Referee'].isnull()]
rows_with_missing_referee.head(7)

,Unnamed: 0.1,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,Away Win Odds,Home Win Odds,Draw Odds,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Wins_Last_4,H2H_Away_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,Unnamed: 0,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A


In [1361]:
# Fill missing 'Referee' values with the mode (most common referee)
mode_referee = table_df['Referee'].mode()[0]
table_df['Referee'].fillna(mode_referee, inplace=True)

# Verify there are no missing values in 'Referee'
print(f"Remaining missing values in 'Referee': {table_df['Referee'].isnull().sum()}")


Remaining missing values in 'Referee': 0


In [1362]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    149
Home Win Odds    149
Draw Odds        149
Temperature       19
dtype: int64


In [1363]:
from sklearn.impute import KNNImputer
import pandas as pd

# Load the dataset (assuming it's in table_df)
# Replace 'Temperature' and any other relevant columns for KNN imputation
columns_to_impute = ['Temperature']  # Include columns correlated with Temperature

# Select only the relevant columns for imputation
impute_df = table_df[columns_to_impute]

# Initialize the KNN imputer
knn_imputer = KNNImputer(n_neighbors=5)  # Use 5 neighbors by default

# Perform KNN imputation
imputed_data = knn_imputer.fit_transform(impute_df)

# Replace the original 'Temperature' column with the imputed values
table_df['Temperature'] = imputed_data[:, 0]

# Verify the imputation
print("Remaining missing values in 'Temperature':", table_df['Temperature'].isnull().sum())


Remaining missing values in 'Temperature': 0


In [1364]:
check_missing_values()

Columns with Missing Values:
Away Win Odds    149
Home Win Odds    149
Draw Odds        149
dtype: int64


In [1365]:
columns_to_remove = ['Away Win Odds', 'Home Win Odds', 'Draw Odds', 'Unnamed: 0.1','Unnamed: 0']
table_df.drop(columns=columns_to_remove, inplace=True)


In [1366]:
check_missing_values()

Columns with Missing Values:
Series([], dtype: int64)


## Check for outliers

In [1368]:
# Function to identify outliers based on IQR
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)  # First quartile
    Q3 = df[column].quantile(0.75)  # Third quartile
    IQR = Q3 - Q1  # Interquartile range
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers

# Check for outliers in numerical columns
numerical_columns = table_df.select_dtypes(include=['float64', 'int64']).columns
outliers = {col: detect_outliers_iqr(table_df, col) for col in numerical_columns}

# Display the columns with outliers and their counts
outlier_counts = {col: len(outliers[col]) for col in outliers if len(outliers[col]) > 0}
print("Columns with outliers and their counts:")
print(outlier_counts)


Columns with outliers and their counts:
{'Home Score': 13, 'Away Score': 7, 'Prev Home Lost': 2, 'Prev Home Goal Difference': 3, 'Prev Away Won': 2, 'Prev Away Lost': 3, 'Prev Away Goals Conceded': 2, 'Prev Away Goal Difference': 6, 'Prev Away Points': 1, 'xG': 4, 'xG.1': 9, 'Temperature': 9, 'H2H_Home_Wins_Last_4': 5, 'H2H_Away_Wins_Last_4': 5, 'H2H_Draws_Last_4': 2, 'home_xg': 2, 'away_xg': 4, 'away_np_xg': 3, 'away_np_xg_difference': 4, 'away_ppda': 7, 'away_deep_completions': 6, 'home_np_xg': 2, 'home_np_xg_difference': 4, 'home_ppda': 8, 'home_deep_completions': 4, 'B365H': 13, 'B365D': 11, 'B365A': 9}


Append the first 10 upcoming fixtures

In [1370]:
# Append the first 10 rows of null_scores_df to table_df using concat
table_df = pd.concat([table_df, null_scores_df.head(10)], ignore_index=True)

In [1371]:
table_df.tail(12)

,Match ID,Home Team,Away Team,Home Score,Away Score,Winner,Prev Home Played,Prev Home Won,Prev Home Drawn,Prev Home Lost,Prev Home Goals Scored,Prev Home Goals Conceded,Prev Home Goal Difference,Prev Home Points,Prev Home Form,Prev Away Played,Prev Away Won,Prev Away Drawn,Prev Away Lost,Prev Away Goals Scored,Prev Away Goals Conceded,Prev Away Goal Difference,Prev Away Points,Prev Away Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Home_Wins_Last_4,H2H_Away_Wins_Last_4,H2H_Draws_Last_4,Composite Key,away_team_code,home_team_code,home_xg,away_xg,is_result,has_data,url,away_points,away_expected_points,away_np_xg,away_np_xg_difference,away_ppda,away_deep_completions,home_points,home_expected_points,home_np_xg,home_np_xg_difference,home_ppda,home_deep_completions,gameweek,home_team_fdr,away_team_fdr,home_team,away_team,composite_key,HomeTeam,AwayTeam,B365H,B365D,B365A,Unnamed: 0.1,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Unnamed: 0
147,497558,Tottenham,Chelsea,3.0,4.0,AWAY_TEAM,14,6,2,6,28,15,13,20,1.4,14,8,4,2,31,15,16,28,2.2,2024-12-08,16:30,15,Sun,2.8,2.8,61184.0,Tottenham Hotspur Stadium,Anthony Taylor,16.0,51.6047,-0.0664,2024-12-08T16:30:00,15.206185,0,0,0,2024-12-08_Tottenham_Chelsea,CHE,TOT,3.25488,2.57326,True,True,https://understat.com/match/26745,3.0,0.9833,1.05092,-2.20396,7.958333,16.0,0.0,1.8101,3.25488,2.20396,9.857143,8.0,15.0,4,4,Tottenham,Chelsea,2024-12-08_Tottenham_Chelsea,Tottenham,Chelsea,3.4,3.9,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,497559,West Ham,Wolves,2.0,1.0,HOME_TEAM,14,4,3,7,18,27,-9,15,0.8,14,2,3,9,22,36,-14,9,1.4,2024-12-09,20:00,15,Mon,1.0,1.4,62474.0,London Stadium,John Brooks,7.0,51.5386,-0.0164,2024-12-09T20:00:00,15.206185,0,0,0,2024-12-09_West Ham_Wolves,WOL,WHU,1.19808,1.47567,True,True,https://understat.com/match/26746,0.0,1.5825,1.47567,0.27759,8.037037,6.0,3.0,1.1481,1.19808,-0.27759,9.043478,7.0,15.0,2,2,West Ham,Wolves,2024-12-09_West Ham_Wolves,West Ham,Wolves,1.9,3.9,3.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,497561,Arsenal,Everton,NaN,NaN,NaN,15,8,5,2,29,15,14,29,2.2,14,3,5,6,14,21,-7,14,1.0,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,Emirates Stadium,NaN,3.0,51.5550,-0.1086,2024-12-14T15:00:00,NaN,0,0,0,2024-12-14_Arsenal_Everton,EVE,ARS,NaN,NaN,False,False,https://understat.com/match/26755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,2,5,Arsenal,Everton,2024-12-14_Arsenal_Everton,NaN,NaN,NaN,NaN,NaN,149.0,NaN,13.0,1.20,5.50,NaN,150.0
150,497564,Liverpool,Fulham,NaN,NaN,NaN,14,11,2,1,29,11,18,35,2.6,15,6,5,4,22,20,2,23,1.6,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,Anfield,NaN,20.0,53.4308,-2.9608,2024-12-14T15:00:00,NaN,0,0,0,2024-12-14_Liverpool_Fulham,FLH,LIV,NaN,NaN,False,False,https://understat.com/match/26752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,2,5,Liverpool,Fulham,2024-12-14_Liverpool_Fulham,NaN,NaN,NaN,NaN,NaN,150.0,NaN,9.0,1.29,5.50,NaN,151.0
151,497566,Newcastle Utd,Leicester City,NaN,NaN,NaN,15,5,5,5,19,21,-2,20,1.0,15,3,5,7,21,30,-9,14,0.8,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,St James' Park,NaN,5.0,54.9756,-1.6217,2024-12-14T15:00:00,NaN,0,0,0,2024-12-14_Newcastle Utd_Leicester City,LEI,NEW,NaN,NaN,False,False,https://understat.com/match/26760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,2,4,Newcastle Utd,Leicester City,2024-12-14_Newcastle Utd_Leicester City,NaN,NaN,NaN,NaN,NaN,151.0,NaN,NaN,NaN,NaN,NaN,152.0
152,497569,Wolves,Ipswich Town,NaN,NaN,NaN,15,2,3,10,23,38,-15,9,1.2,15,1,6,8,14,27,-13,9,0.8,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,Molineux Stadium,NaN,19.0,52.5903,-2.1303,2024-12-14T15:00:00,NaN,0,0,0,2024-12-14_Wolves_Ipswich Town,IPS,WOL,NaN,NaN,False,False,https://understat.com/match/26758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,2,2,Wolves,Ipswich Town,2024-12-14_Wolves_Ipswich Town,NaN,NaN,NaN,NaN,NaN,152.0,NaN,NaN,NaN,NaN,NaN,153.0
153,497567,Nottingham Forest,Aston Villa,NaN,NaN,NaN,15,7,4,4,19,18,1,25,1.2,15,7,4,4,23,23,0,25,1.4,2024-12-14,17:30,16,Sat,NaN,

# Feature Engineering

### Column names changed for feature engineering

In [1374]:
import pandas as pd

data = table_df
# Mapping the old column names to new names where 'Home' and 'Away' are replaced with 'Team 1' and 'Team 2'
column_mapping = {
    'Home Team': 'Team 1',
    'Away Team': 'Team 2',
    'Home Score': 'Team 1 Score',
    'Away Score': 'Team 2 Score',
    'Prev Home Played': 'Prev Team 1 Played',
    'Prev Away Played': 'Prev Team 2 Played',
    'Prev Home Won': 'Prev Team 1 Won',
    'Prev Home Drawn': 'Prev Team 1 Drawn',
    'Prev Home Lost': 'Prev Team 1 Lost',
    'Prev Home Goals Scored': 'Prev Team 1 Goals Scored',
    'Prev Home Goals Conceded': 'Prev Team 1 Goals Conceded',
    'Prev Home Goal Difference': 'Prev Team 1 Goal Difference',
    'Prev Home Points': 'Prev Team 1 Points',
    'Prev Home Form': 'Prev Team 1 Form',
    'Prev Away Won': 'Prev Team 2 Won',
    'Prev Away Drawn': 'Prev Team 2 Drawn',
    'Prev Away Lost': 'Prev Team 2 Lost',
    'Prev Away Goals Scored': 'Prev Team 2 Goals Scored',
    'Prev Away Goals Conceded': 'Prev Team 2 Goals Conceded',
    'Prev Away Goal Difference': 'Prev Team 2 Goal Difference',
    'Prev Away Points': 'Prev Team 2 Points',
    'Prev Away Form': 'Prev Team 2 Form',
    'H2H_Home_Wins_Last_4': 'H2H_Team_1_Wins_Last_4',
    'H2H_Away_Wins_Last_4': 'H2H_Team_2_Wins_Last_4',
    'H2H_Draws_Last_4': 'H2H_Draws_Last_4',
    'home_team_code': 'team_1_code',
    'away_team_code': 'team_2_code',
    'home_xg': 'team_1_xg',
    'away_xg': 'team_2_xg',
    'home_team': 'team_1',
    'away_team': 'team_2',
    'HomeTeam': 'Team1',
    'AwayTeam': 'Team2',
    'home_team_fdr': 'team_1_fdr',
    'away_team_fdr': 'team_2_fdr',
    'home_deep_completions': 'team1_deep_completions',
    'away_deep_completions': 'team2_deep_completions',
    'away_np_xg': 'team2_np_xg',
    'home_np_xg': 'team1_np_xg',
    'away_np_xg_difference':'team2_np_xg_difference',
    'home_np_xg_difference':'home_np_xg_difference',
    'away_np_xg': 'team2_np_xg',
    'home_ppda':'team1_ppda', 
    'away_ppda': 'team2_ppda',
    'home_points': 'team1_points',
    'away_points': 'team2_points',
    'away_expected_points': 'team2_expected_points',
    'home_expected_points':'team1_expected_points',
    'home_np_xg_difference': 'team1_np_xg_difference'
}

# Renaming the columns in the dataframe
data.rename(columns=column_mapping, inplace=True)
data.columns


Index(['Match ID', 'Team 1', 'Team 2', 'Team 1 Score', 'Team 2 Score',
       'Winner', 'Prev Team 1 Played', 'Prev Team 1 Won', 'Prev Team 1 Drawn',
       'Prev Team 1 Lost', 'Prev Team 1 Goals Scored',
       'Prev Team 1 Goals Conceded', 'Prev Team 1 Goal Difference',
       'Prev Team 1 Points', 'Prev Team 1 Form', 'Prev Team 2 Played',
       'Prev Team 2 Won', 'Prev Team 2 Drawn', 'Prev Team 2 Lost',
       'Prev Team 2 Goals Scored', 'Prev Team 2 Goals Conceded',
       'Prev Team 2 Goal Difference', 'Prev Team 2 Points', 'Prev Team 2 Form',
       'Date', 'Time', 'Wk', 'Day', 'xG', 'xG.1', 'Attendance', 'Venue',
       'Referee', 'StadiumID', 'Latitude', 'Longitude', 'Datetime',
       'Temperature', 'H2H_Team_1_Wins_Last_4', 'H2H_Team_2_Wins_Last_4',
       'H2H_Draws_Last_4', 'Composite Key', 'team_2_code', 'team_1_code',
       'team_1_xg', 'team_2_xg', 'is_result', 'has_data', 'url',
       'team2_points', 'team2_expected_points', 'team2_np_xg',
       'team2_np_xg_differe

In [1375]:
data.tail(12)

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Team_1_Wins_Last_4,H2H_Team_2_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,gameweek,team_1_fdr,team_2_fdr,team_1,team_2,composite_key,Team1,Team2,B365H,B365D,B365A,Unnamed: 0.1,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Unnamed: 0
147,497558,Tottenham,Chelsea,3.0,4.0,AWAY_TEAM,14,6,2,6,28,15,13,20,1.4,14,8,4,2,31,15,16,28,2.2,2024-12-08,16:30,15,Sun,2.8,2.8,61184.0,Tottenham Hotspur Stadium,Anthony Taylor,16.0,51.6047,-0.0664,2024-12-08T16:30:00,15.206185,0,0,0,2024-12-08_Tottenham_Chelsea,CHE,TOT,3.25488,2.57326,True,True,https://understat.com/match/26745,3.0,0.9833,1.05092,-2.20396,7.958333,16.0,0.0,1.8101,3.25488,2.20396,9.857143,8.0,15.0,4,4,Tottenham,Chelsea,2024-12-08_Tottenham_Chelsea,Tottenham,Chelsea,3.4,3.9,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,497559,West Ham,Wolves,2.0,1.0,HOME_TEAM,14,4,3,7,18,27,-9,15,0.8,14,2,3,9,22,36,-14,9,1.4,2024-12-09,20:00,15,Mon,1.0,1.4,62474.0,London Stadium,John Brooks,7.0,51.5386,-0.0164,2024-12-09T20:00:00,15.206185,0,0,0,2024-12-09_West Ham_Wolves,WOL,WHU,1.19808,1.47567,True,True,https://understat.com/match/26746,0.0,1.5825,1.47567,0.27759,8.037037,6.0,3.0,1.1481,1.19808,-0.27759,9.043478,7.0,15.0,2,2,West Ham,Wolves,2024-12-09_West Ham_Wolves,West Ham,Wolves,1.9,3.9,3.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN
149,497561,Arsenal,Everton,NaN,NaN,NaN,15,8,5,2,29,15,14,29,2.2,14,3,5,6,14,21,-7,14,1.0,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,Emirates Stadium,NaN,3.0,51.5550,-0.1086,2024-12-14T15:00:00,NaN,0,0,0,2024-12-14_Arsenal_Everton,EVE,ARS,NaN,NaN,False,False,https://understat.com/match/26755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,2,5,Arsenal,Everton,2024-12-14_Arsenal_Everton,NaN,NaN,NaN,NaN,NaN,149.0,NaN,13.0,1.20,5.50,NaN,150.0
150,497564,Liverpool,Fulham,NaN,NaN,NaN,14,11,2,1,29,11,18,35,2.6,15,6,5,4,22,20,2,23,1.6,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,Anfield,NaN,20.0,53.4308,-2.9608,2024-12-14T15:00:00,NaN,0,0,0,2024-12-14_Liverpool_Fulham,FLH,LIV,NaN,NaN,False,False,https://understat.com/match/26752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,2,5,Liverpool,Fulham,2024-12-14_Liverpool_Fulham,NaN,NaN,NaN,NaN,NaN,150.0,NaN,9.0,1.29,5.50,NaN,151.0
151,497566,Newcastle Utd,Leicester City,NaN,NaN,NaN,15,5,5,5,19,21,-2,20,1.0,15,3,5,7,21,30,-9,14,0.8,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,St James' Park,NaN,5.0,54.9756,-1.6217,2024-12-14T15:00:00,NaN,0,0,0,2024-12-14_Newcastle Utd_Leicester City,LEI,NEW,NaN,NaN,False,False,https://understat.com/match/26760,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,2,4,Newcastle Utd,Leicester City,2024-12-14_Newcastle Utd_Leicester City,NaN,NaN,NaN,NaN,NaN,151.0,NaN,NaN,NaN,NaN,NaN,152.0
152,497569,Wolves,Ipswich Town,NaN,NaN,NaN,15,2,3,10,23,38,-15,9,1.2,15,1,6,8,14,27,-13,9,0.8,2024-12-14,15:00,16,Sat,NaN,NaN,NaN,Molineux Stadium,NaN,19.0,52.5903,-2.1303,2024-12-14T15:00:00,NaN,0,0,0,2024-12-14_Wolves_Ipswich Town,IPS,WOL,NaN,NaN,False,False,https://understat.com/match/26758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,2,2,Wolves,Ipswich Town,2024-12-14_Wolves_Ipswich Town,NaN,NaN,NaN,NaN,NaN,152.0,NaN,NaN,NaN,NaN,NaN,153.0
153,497567,Nottingham Forest,Aston Villa,NaN,NaN,NaN,15,7,4,4,19,18,1,25,1.2,15,7,4,4,23,23,0,25,1.

### FE: Average fdr

In [1377]:
def calculate_fdr_cumulative_features(dataframe):
    # Initialize dictionaries to track cumulative sums and games played
    cumulative_fdr = {}
    games_played = {}
    
    # Create new columns for average FDR and cumulative FDR
    avg_team_1_fdr = []
    avg_team_2_fdr = []
    cum_team_1_fdr = []
    cum_team_2_fdr = []

    # Iterate over rows
    for index, row in dataframe.iterrows():
        team_1 = row["Team 1"]
        team_2 = row["Team 2"]
        
        # Initialize if not present in dictionary
        if team_1 not in cumulative_fdr:
            cumulative_fdr[team_1] = 0
            games_played[team_1] = 0
        if team_2 not in cumulative_fdr:
            cumulative_fdr[team_2] = 0
            games_played[team_2] = 0

        # Update cumulative FDR
        cumulative_fdr[team_1] += row['team_1_fdr']
        cumulative_fdr[team_2] += row['team_2_fdr']
        
        # Increment games played
        games_played[team_1] += 1
        games_played[team_2] += 1

         # Record cumulative FDR for the current row
        cum_team_1_fdr.append(cumulative_fdr[team_1])
        cum_team_2_fdr.append(cumulative_fdr[team_2])
        
        # Calculate averages for teams before updating current match FDR
       
        avg_team_1_fdr.append(cumulative_fdr[team_1] / (games_played[team_1])) 
        avg_team_2_fdr.append(cumulative_fdr[team_2] / (games_played[team_2]))

    # Add new columns to the DataFrame
    dataframe['Cum Team 1 FDR'] = cum_team_1_fdr
    dataframe['Cum Team 2 FDR'] = cum_team_2_fdr
    dataframe['Avg Team 1 FDR'] = avg_team_1_fdr
    dataframe['Avg Team 2 FDR'] = avg_team_2_fdr

    

    return dataframe

# Calculate cumulative features excluding current match
detailed_fixtures = calculate_fdr_cumulative_features(data)

# Display selected rows with cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'team_1_fdr', 'team_2_fdr',
    'Cum Team 1 FDR', 'Cum Team 2 FDR', 'Avg Team 1 FDR', 'Avg Team 2 FDR'
]



In [1378]:
detailed_fixtures[['Team 1','Team 2','Prev Team 1 Played',
'Prev Team 2 Played','Avg Team 1 FDR','Avg Team 2 FDR','team_1_fdr', 'team_2_fdr','Cum Team 1 FDR','Cum Team 2 FDR']].to_csv("detailed_fixture.csv")

In [1379]:
table_df.head()

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Team_1_Wins_Last_4,H2H_Team_2_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,gameweek,team_1_fdr,team_2_fdr,team_1,team_2,composite_key,Team1,Team2,B365H,B365D,B365A,Unnamed: 0.1,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Unnamed: 0,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,3,1,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1.0,2,3,Manchester Utd,Fulham,2024-08-16_Manchester Utd_Fulham,Manchester Utd,Fulham,1.60,4.20,5.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,2.0,3.0
1,497411,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,11:30,1,Sat,0.5,2.6,30014.0,Portman Road Stadium,Tim Robinson,2.0,52.0567,1.1450,2024-08-17T11:30:00,22.212500,0,0,0,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,1.0,5,2,Ipswich Town,Liverpool,2024-08-17_Ipswich Town_Liverpool,Ipswich Town,Liverpool,8.50,5.50,1.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2,5.0,2.0
2,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,4,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,1.0,2,5,Arsenal,Wolves,2024-08-17_Arsenal_Wolves,Arsenal,Wolves,1.18,7.50,13.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,5,2.0,5.0
3,497413,Everton,Brighton,0.0,3.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.5,1.4,39217.0,Goodison Park,Simon Hooper,4.0,53.4389,-2.9664,2024-08-17T14:00:00,17.587500,0,0,0,2024-08-17_Everton_Brighton,BRI,EVE,0.405325,1.790830,True,True,https://understat.com/match/26605,3.0,2.5287,1.790830,1.385505,7.916667,5.0,0.0,0.3078,0.405325,-1.385505,18.333333,4.0,1.0,2,2,Everton,Brighton,2024-08-17_Everton_Brighton,Everton,Brighton,2.63,3.30,2.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2.0,2.0
4,497414,Newcastle Utd,Southampton,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.3,1.8,52196.0,St James' Park,Craig Pawson,5.0,54.9756,-1.6217,2024-08-17T14:00:00,18.622499,0,0,0,2024-08-17_Newcastle Utd_Southampton,SOU,NEW,0.433489,1.954830,True,True,https://understat.com/match/26606,0.0,2.5586,1.954830,1.521341,3.789474,13.0,3.0,0.2774,0.433489,-1.521341,16.250000,4.0,1.0,2,4,Newcastle Utd,Southampton,2024-08-17_Newcastle Utd_Southampton,Newcastle Utd,Southampton,1.36,5.25,8.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,4,2.0,4.0


In [1380]:
table_df = table_df.drop(columns=['team_1', 'team_2','Team1', 'Team2'])

In [1381]:
table_df.to_csv("fact_table.csv")

In [1382]:
df = table_df

In [1383]:
df.columns

Index(['Match ID', 'Team 1', 'Team 2', 'Team 1 Score', 'Team 2 Score',
       'Winner', 'Prev Team 1 Played', 'Prev Team 1 Won', 'Prev Team 1 Drawn',
       'Prev Team 1 Lost', 'Prev Team 1 Goals Scored',
       'Prev Team 1 Goals Conceded', 'Prev Team 1 Goal Difference',
       'Prev Team 1 Points', 'Prev Team 1 Form', 'Prev Team 2 Played',
       'Prev Team 2 Won', 'Prev Team 2 Drawn', 'Prev Team 2 Lost',
       'Prev Team 2 Goals Scored', 'Prev Team 2 Goals Conceded',
       'Prev Team 2 Goal Difference', 'Prev Team 2 Points', 'Prev Team 2 Form',
       'Date', 'Time', 'Wk', 'Day', 'xG', 'xG.1', 'Attendance', 'Venue',
       'Referee', 'StadiumID', 'Latitude', 'Longitude', 'Datetime',
       'Temperature', 'H2H_Team_1_Wins_Last_4', 'H2H_Team_2_Wins_Last_4',
       'H2H_Draws_Last_4', 'Composite Key', 'team_2_code', 'team_1_code',
       'team_1_xg', 'team_2_xg', 'is_result', 'has_data', 'url',
       'team2_points', 'team2_expected_points', 'team2_np_xg',
       'team2_np_xg_differe

### FE: Average ppda/deep completion

In [1385]:
def calculate_weighted_ppda_deep_completions(df):
    # Initialize dictionaries to track cumulative sums and games played
    cumulative_ppda = {}
    cumulative_deep = {}
    cumulative_fdr = {}
    games_played = {}

    # Create new columns for weighted average PPDA and deep completions
    cum_ppda_team1 = []
    cum_ppda_team2 = []
    cum_deep_team1 = []
    cum_deep_team2 = []
    avg_ppda_team1 = []
    avg_ppda_team2 = []
    avg_deep_team1 = []
    avg_deep_team2 = []
    weighted_avg_ppda_team1 = []
    weighted_avg_ppda_team2 = []
    weighted_avg_deep_team1 = []
    weighted_avg_deep_team2 = []

    # Iterate over rows
    for index, row in df.iterrows():
        team1 = row["Team 1"]
        team2 = row["Team 2"]

        # Initialize if not present in the dictionary
        for team in (team1, team2):
            if team not in cumulative_ppda:
                cumulative_ppda[team] = 0
                cumulative_deep[team] = 0
                cumulative_fdr[team] = 0
                games_played[team] = 0

        # Record cumulative values for the current row before updating them
        cum_ppda_team1.append(cumulative_ppda[team1])
        cum_ppda_team2.append(cumulative_ppda[team2])
        cum_deep_team1.append(cumulative_deep[team1])
        cum_deep_team2.append(cumulative_deep[team2])

        # Calculate averages based on games played before this match
        if games_played[team1] > 0:
            avg_ppda_team1.append(cumulative_ppda[team1] / games_played[team1])
            avg_deep_team1.append(cumulative_deep[team1] / games_played[team1])
            weighted_avg_ppda_team1.append((cumulative_ppda[team1] / games_played[team1]) * (cumulative_fdr[team1] / games_played[team1]))
            weighted_avg_deep_team1.append((cumulative_deep[team1] / games_played[team1]) * (cumulative_fdr[team1] / games_played[team1]))
        else:
            avg_ppda_team1.append(0)
            avg_deep_team1.append(0)
            weighted_avg_ppda_team1.append(0)
            weighted_avg_deep_team1.append(0)

        if games_played[team2] > 0:
            avg_ppda_team2.append(cumulative_ppda[team2] / games_played[team2])
            avg_deep_team2.append(cumulative_deep[team2] / games_played[team2])
            weighted_avg_ppda_team2.append((cumulative_ppda[team2] / games_played[team2]) * (cumulative_fdr[team2] / games_played[team2]))
            weighted_avg_deep_team2.append((cumulative_deep[team2] / games_played[team2]) * (cumulative_fdr[team2] / games_played[team2]))
        else:
            avg_ppda_team2.append(0)
            avg_deep_team2.append(0)
            weighted_avg_ppda_team2.append(0)
            weighted_avg_deep_team2.append(0)

        # Update cumulative PPDA, deep completions, and FDR after recording the current match
        cumulative_ppda[team1] += row['team1_ppda']
        cumulative_ppda[team2] += row['team2_ppda']
        cumulative_deep[team1] += row['team1_deep_completions']
        cumulative_deep[team2] += row['team2_deep_completions']
        cumulative_fdr[team1] += row['team_1_fdr']
        cumulative_fdr[team2] += row['team_2_fdr']

        # Increment games played after using the counts
        games_played[team1] += 1
        games_played[team2] += 1

    # Add new columns to the DataFrame
    df['Cum PPDA Team 1'] = cum_ppda_team1
    df['Cum PPDA Team 2'] = cum_ppda_team2
    df['Avg PPDA Team 1'] = avg_ppda_team1
    df['Avg PPDA Team 2'] = avg_ppda_team2
    df['Cum Deep Completions Team 1'] = cum_deep_team1
    df['Cum Deep Completions Team 2'] = cum_deep_team2
    df['Avg Deep Completions Team 1'] = avg_deep_team1
    df['Avg Deep Completions Team 2'] = avg_deep_team2
    df['Weighted Avg PPDA Team 1'] = weighted_avg_ppda_team1
    df['Weighted Avg PPDA Team 2'] = weighted_avg_ppda_team2
    df['Weighted Avg Deep Completions Team 1'] = weighted_avg_deep_team1
    df['Weighted Avg Deep Completions Team 2'] = weighted_avg_deep_team2

    return df

# Example usage with your dataset
updated_data = calculate_weighted_ppda_deep_completions(df)

# Display selected rows with weighted cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'team1_ppda', 'team2_ppda',
    'Weighted Avg PPDA Team 1', 'Weighted Avg PPDA Team 2',
    'Weighted Avg Deep Completions Team 1', 'Weighted Avg Deep Completions Team 2'
]
display_df = updated_data[columns_to_display]


In [1386]:
display_df.head(12)

,Match ID,Team 1,Team 2,team1_ppda,team2_ppda,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2
0,497410,Manchester Utd,Fulham,7.379310,10.833333,0.000000,0.000000,0.0,0.0
1,497411,Ipswich Town,Liverpool,18.777778,8.739130,0.000000,0.000000,0.0,0.0
2,497412,Arsenal,Wolves,7.769231,10.818182,0.000000,0.000000,0.0,0.0
3,497413,Everton,Brighton,18.333333,7.916667,0.000000,0.000000,0.0,0.0
4,497414,Newcastle Utd,Southampton,16.250000,3.789474,0.000000,0.000000,0.0,0.0
5,497415,Nottingham Forest,Bournemouth,8.653846,9.954545,0.000000,0.000000,0.0,0.0
6,497416,West Ham,Aston Villa,9.333333,11.304348,0.000000,0.000000,0.0,0.0
7,497417,Brentford,Crystal Palace,12.666667,7.724138,0.000000,0.000000,0.0,0.0
8,497418,Chelsea,Manchester City,10.064516,15.058824,0.000000,0.000000,0.0,0.0
9,497419,Leicester City,Tottenham,20.687500,5.482759,0.000000,0.000000,0.0,0.0


In [1387]:
updated_data.head(12)

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Team_1_Wins_Last_4,H2H_Team_2_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,Unnamed: 0.1,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Unnamed: 0,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,3,1,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1.0,2,3,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,2.0,3.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
1,497411,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,11:30,1,Sat,0.5,2.6,30014.0,Portman Road Stadium,Tim Robinson,2.0,52.0567,1.1450,2024-08-17T11:30:00,22.212500,0,0,0,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,1.0,5,2,2024-08-17_Ipswich Town_Liverpool,8.50,5.50,1.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2,5.0,2.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
2,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,4,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,1.0,2,5,2024-08-17_Arsenal_Wolves,1.18,7.50,13.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,5,2.0,5.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,497413,Everton,Brighton,0.0,3.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.5,1.4,39217.0,Goodison Park,Simon Hooper,4.0,53.4389,-2.9664,2024-08-17T14:00:00,17.587500,0,0,0,2024-08-17_Everton_Brighton,BRI,EVE,0.405325,1.790830,True,True,https://understat.com/match/26605,3.0,2.5287,1.790830,1.385505,7.916667,5.0,0.0,0.3078,0.405325,-1.385505,18.333333,4.0,1.0,2,2,2024-08-17_Everton_Brighton,2.63,3.30,2.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2.0,2.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,497414,Newcastle Utd,Southampton,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.3,1.8,52196.0,St James' Park,Craig Pawson,5.0,54.9756,-1.6217,2024-08-17T14:00:00,18.622499,0,0,0,2024-08-17_Newcastle Utd_Southampton,SOU,NEW,0.433489,1.954830,True,True,https://understat.com/match/26606,0.0,2.5586,1.95

In [1388]:
updated_data.columns

Index(['Match ID', 'Team 1', 'Team 2', 'Team 1 Score', 'Team 2 Score',
       'Winner', 'Prev Team 1 Played', 'Prev Team 1 Won', 'Prev Team 1 Drawn',
       'Prev Team 1 Lost', 'Prev Team 1 Goals Scored',
       'Prev Team 1 Goals Conceded', 'Prev Team 1 Goal Difference',
       'Prev Team 1 Points', 'Prev Team 1 Form', 'Prev Team 2 Played',
       'Prev Team 2 Won', 'Prev Team 2 Drawn', 'Prev Team 2 Lost',
       'Prev Team 2 Goals Scored', 'Prev Team 2 Goals Conceded',
       'Prev Team 2 Goal Difference', 'Prev Team 2 Points', 'Prev Team 2 Form',
       'Date', 'Time', 'Wk', 'Day', 'xG', 'xG.1', 'Attendance', 'Venue',
       'Referee', 'StadiumID', 'Latitude', 'Longitude', 'Datetime',
       'Temperature', 'H2H_Team_1_Wins_Last_4', 'H2H_Team_2_Wins_Last_4',
       'H2H_Draws_Last_4', 'Composite Key', 'team_2_code', 'team_1_code',
       'team_1_xg', 'team_2_xg', 'is_result', 'has_data', 'url',
       'team2_points', 'team2_expected_points', 'team2_np_xg',
       'team2_np_xg_differe

### FE: Avg xg stats

In [1390]:
df.head()

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Team_1_Wins_Last_4,H2H_Team_2_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,Unnamed: 0.1,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Unnamed: 0,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2
0,497410,Manchester Utd,Fulham,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-16,19:00,1,Fri,2.4,0.4,73297.0,Old Trafford,Robert Jones,1.0,53.4631,-2.2914,2024-08-16T19:00:00,14.313000,3,1,0,2024-08-16_Manchester Utd_Fulham,FLH,MUN,2.042680,0.418711,True,True,https://understat.com/match/26602,0.0,0.2815,0.418711,-1.623969,10.833333,3.0,3.0,2.5696,2.042680,1.623969,7.379310,7.0,1.0,2,3,2024-08-16_Manchester Utd_Fulham,1.60,4.20,5.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,3,2.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,497411,Ipswich Town,Liverpool,0.0,2.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,11:30,1,Sat,0.5,2.6,30014.0,Portman Road Stadium,Tim Robinson,2.0,52.0567,1.1450,2024-08-17T11:30:00,22.212500,0,0,0,2024-08-17_Ipswich Town_Liverpool,LIV,IPS,0.342601,3.929060,True,True,https://understat.com/match/26603,3.0,2.9562,3.929060,3.586459,8.739130,13.0,0.0,0.0258,0.342601,-3.586459,18.777778,2.0,1.0,5,2,2024-08-17_Ipswich Town_Liverpool,8.50,5.50,1.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,2,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,497412,Arsenal,Wolves,2.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,1.2,0.5,60261.0,Emirates Stadium,Jarred Gillett,3.0,51.5550,-0.1086,2024-08-17T14:00:00,21.734999,4,0,0,2024-08-17_Arsenal_Wolves,WOL,ARS,1.628300,0.575835,True,True,https://understat.com/match/26604,0.0,0.5796,0.575835,-1.052465,10.818182,2.0,3.0,2.1894,1.628300,1.052465,7.769231,14.0,1.0,2,5,2024-08-17_Arsenal_Wolves,1.18,7.50,13.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,5,2.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,497413,Everton,Brighton,0.0,3.0,AWAY_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.5,1.4,39217.0,Goodison Park,Simon Hooper,4.0,53.4389,-2.9664,2024-08-17T14:00:00,17.587500,0,0,0,2024-08-17_Everton_Brighton,BRI,EVE,0.405325,1.790830,True,True,https://understat.com/match/26605,3.0,2.5287,1.790830,1.385505,7.916667,5.0,0.0,0.3078,0.405325,-1.385505,18.333333,4.0,1.0,2,2,2024-08-17_Everton_Brighton,2.63,3.30,2.63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,497414,Newcastle Utd,Southampton,1.0,0.0,HOME_TEAM,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0.0,2024-08-17,14:00,1,Sat,0.3,1.8,52196.0,St James' Park,Craig Pawson,5.0,54.9756,-1.6217,2024-08-17T14:00:00,18.622499,0,0,0,2024-08-17_Newcastle Utd_Southampton,SOU,NEW,0.433489,1.954830,True,True,https://understat.com/match/26606,0.0,2.5586,1.954830,1.521341,3.789474,13.0,3.0,0.2774,0.433489,-1.521341,16.250000,4.0,1.0,2,4,2024-08-17_Newcastle Utd_Southampton,1.3

In [1391]:
def calculate_weighted_xg_stats(df):
    # Initialize dictionaries to track cumulative sums and games played
    xg_stats = [
        'xG', 'xG.1',
    ]
    cumulative = {stat: {} for stat in xg_stats}
    games_played = {}

    # Initialize columns for cumulative, average, and weighted average stats
    for stat in xg_stats:
        df[f'Cum {stat}'] = 0
        df[f'Avg {stat}'] = 0.0
        df[f'Weighted Avg {stat}'] = 0.0

    # Iterate over rows
    for index, row in df.iterrows():
        team1 = row["Team 1"]
        team2 = row["Team 2"]

        # Initialize if not present in the dictionary
        for team in [team1, team2]:
            if team not in games_played:
                games_played[team] = 0
                for stat in xg_stats:
                    cumulative[stat][team] = 0

        # Compute averages for each stat based on previous matches' data
        for stat in xg_stats:
            team_key = team1 if '.1' not in stat and 'team2' not in stat else team2
            corrected_stat = stat.replace('.1', '') if '.1' in stat else stat  # Adjust the stat key for team2

            # Before updating the cumulative, record the current game's value
            value_to_add = row[stat] if pd.notna(row[stat]) else 0

            # Calculate averages using prior match data
            if games_played[team_key] >= 1:  # Now checks if there's at least one past game
                current_avg = cumulative[corrected_stat][team_key] / games_played[team_key]
                df.at[index, f'Avg {stat}'] = current_avg
                fdr = row['Avg Team 1 FDR'] if team_key == team1 else row['Avg Team 2 FDR']
                df.at[index, f'Weighted Avg {stat}'] = current_avg * fdr
            else:
                df.at[index, f'Avg {stat}'] = 0
                df.at[index, f'Weighted Avg {stat}'] = 0

            # Update cumulative values after computing averages
            cumulative[corrected_stat][team_key] += value_to_add

        # Increment games played after updating stats
        games_played[team1] += 1
        games_played[team2] += 1

    return df

In [1392]:
# Example usage with your dataset
data_df = calculate_weighted_xg_stats(df)

# Display selected rows with weighted cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'xG', 'xG.1','Avg xG','Avg xG.1', 'Weighted Avg xG', 'Weighted Avg xG.1'
]
display_df = data_df[columns_to_display]


In [1477]:
display_df.head()

,Match ID,Team 1,Team 2,xG,xG.1,Avg xG,Avg xG.1,Weighted Avg xG,Weighted Avg xG.1
0,497410,Manchester Utd,Fulham,2.4,0.4,0.0,0.0,0.0,0.0
1,497411,Ipswich Town,Liverpool,0.5,2.6,0.0,0.0,0.0,0.0
2,497412,Arsenal,Wolves,1.2,0.5,0.0,0.0,0.0,0.0
3,497413,Everton,Brighton,0.5,1.4,0.0,0.0,0.0,0.0
4,497414,Newcastle Utd,Southampton,0.3,1.8,0.0,0.0,0.0,0.0


In [1479]:
def calculate_weighted_xg_stats(df):
    # Initialize dictionaries to track cumulative sums and games played
    cumulative_xg = {}
    games_played = {}

    # Create new columns for xG stats
    cum_xg_team1 = []
    cum_xg_team2 = []
    avg_xg_team1 = []
    avg_xg_team2 = []
    weighted_avg_xg_team1 = []
    weighted_avg_xg_team2 = []

    # Iterate over rows
    for index, row in df.iterrows():
        team1 = row["Team 1"]
        team2 = row["Team 2"]

        # Initialize if not present in the dictionary
        for team in (team1, team2):
            if team not in cumulative_xg:
                cumulative_xg[team] = 0
                games_played[team] = 0

        # Record cumulative values for the current row before updating them
        cum_xg_team1.append(cumulative_xg[team1])
        cum_xg_team2.append(cumulative_xg[team2])

        # Calculate averages based on games played before this match
        if games_played[team1] > 0:
            avg_xg_team1.append(cumulative_xg[team1] / games_played[team1])
            # Multiply the average by 'Avg Team 1 FDR' for weighted average
            weighted_avg_xg_team1.append(avg_xg_team1[-1] * row['Avg Team 1 FDR'])
        else:
            avg_xg_team1.append(0)
            weighted_avg_xg_team1.append(0)

        if games_played[team2] > 0:
            avg_xg_team2.append(cumulative_xg[team2] / games_played[team2])
            # Multiply the average by 'Avg Team 2 FDR' for weighted average
            weighted_avg_xg_team2.append(avg_xg_team2[-1] * row['Avg Team 2 FDR'])
        else:
            avg_xg_team2.append(0)
            weighted_avg_xg_team2.append(0)

        # Update cumulative xG after recording the current match
        cumulative_xg[team1] += row['team_1_xg']
        cumulative_xg[team2] += row['team_2_xg']

        # Increment games played after using the counts
        games_played[team1] += 1
        games_played[team2] += 1

    # Add new columns to the DataFrame
    df['Cum xG Team 1'] = cum_xg_team1
    df['Cum xG Team 2'] = cum_xg_team2
    df['Avg xG Team 1'] = avg_xg_team1
    df['Avg xG Team 2'] = avg_xg_team2
    df['Weighted Avg xG Team 1'] = weighted_avg_xg_team1
    df['Weighted Avg xG Team 2'] = weighted_avg_xg_team2

    return df
    
# Example usage with your dataset
updated_data = calculate_weighted_xg_stats(data_df)

# Display selected rows with weighted cumulative values and averages
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2', 'team_1_xg', 'team_2_xg',
    'Avg xG Team 1', 'Avg xG Team 2',
    'Weighted Avg xG Team 1', 'Weighted Avg xG Team 2',  'xG', 'xG.1','Avg xG','Avg xG.1', 'Weighted Avg xG', 'Weighted Avg xG.1'
]
displayxg_df = updated_data[columns_to_display]


In [1481]:
displayxg_df.head(30)

,Match ID,Team 1,Team 2,team_1_xg,team_2_xg,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,xG,xG.1,Avg xG,Avg xG.1,Weighted Avg xG,Weighted Avg xG.1
0,497410,Manchester Utd,Fulham,2.042680,0.418711,0.000000,0.000000,0.000000,0.000000,2.4,0.4,0.00,0.00,0.000000,0.000000
1,497411,Ipswich Town,Liverpool,0.342601,3.929060,0.000000,0.000000,0.000000,0.000000,0.5,2.6,0.00,0.00,0.000000,0.000000
2,497412,Arsenal,Wolves,1.628300,0.575835,0.000000,0.000000,0.000000,0.000000,1.2,0.5,0.00,0.00,0.000000,0.000000
3,497413,Everton,Brighton,0.405325,1.790830,0.000000,0.000000,0.000000,0.000000,0.5,1.4,0.00,0.00,0.000000,0.000000
4,497414,Newcastle Utd,Southampton,0.433489,1.954830,0.000000,0.000000,0.000000,0.000000,0.3,1.8,0.00,0.00,0.000000,0.000000
5,497415,Nottingham Forest,Bournemouth,1.244050,1.909150,0.000000,0.000000,0.000000,0.000000,1.3,1.2,0.00,0.00,0.000000,0.000000
6,497416,West Ham,Aston Villa,2.804700,2.670300,0.000000,0.000000,0.000000,0.000000,2.3,2.0,0.00,0.00,0.000000,0.000000
7,497417,Brentford,Crystal Palace,1.380640,1.147570,0.000000,0.000000,0.000000,0.000000,1.6,1.2,0.00,0.00,0.000000,0.000000
8,497418,Chelsea,Manchester City,1.061230,1.180380,0.000000,0.000000,0.000000,0.000000,1.0,0.8,0.00,0.00,0.000000,0.000000
9,497419,Leicester City,Tottenham,1.026140,1.407750,0.000000,0.000000,0.000000,0.000000,1.0,1.2,0.00,0.00,0.000000,0.000000


In [1483]:
def calculate_weighted_stats_xp(df):
    stats = ['np_xg', 'expected_points']  # Unified stat names
    cumulative = {stat: {} for stat in stats}  # Unified cumulative stats
    games_played = {}

    # Initialize result columns
    for stat in stats:
        df[f'Cum {stat}'] = 0  # Unified cumulative for each team
        df[f'Avg team1_{stat}'] = 0.0
        df[f'Avg team2_{stat}'] = 0.0
        df[f'Weighted Avg team1_{stat}'] = 0.0
        df[f'Weighted Avg team2_{stat}'] = 0.0

    for index, row in df.iterrows():
        team1 = row["Team 1"]
        team2 = row["Team 2"]

        # Initialize teams in cumulative and games_played if not already present
        for team in [team1, team2]:
            if team not in games_played:
                games_played[team] = 0
                for stat in stats:
                    cumulative[stat][team] = 0.0

        for stat in stats:
            # Process for team1
            if games_played[team1] > 0:  # Exclude current match stats
                avg_team1 = cumulative[stat][team1] / games_played[team1]
                fdr_team1 = row['Avg Team 1 FDR']
                weighted_avg_team1 = avg_team1 * fdr_team1

                df.at[index, f'Avg team1_{stat}'] = avg_team1
                df.at[index, f'Weighted Avg team1_{stat}'] = weighted_avg_team1
            else:
                df.at[index, f'Avg team1_{stat}'] = 0
                df.at[index, f'Weighted Avg team1_{stat}'] = 0

            # Process for team2
            if games_played[team2] > 0:  # Exclude current match stats
                avg_team2 = cumulative[stat][team2] / games_played[team2]
                fdr_team2 = row['Avg Team 2 FDR']
                weighted_avg_team2 = avg_team2 * fdr_team2

                df.at[index, f'Avg team2_{stat}'] = avg_team2
                df.at[index, f'Weighted Avg team2_{stat}'] = weighted_avg_team2
            else:
                df.at[index, f'Avg team2_{stat}'] = 0
                df.at[index, f'Weighted Avg team2_{stat}'] = 0

            # Update cumulative stats AFTER calculating averages
            team1_value = row[f'team1_{stat}'] if pd.notna(row[f'team1_{stat}']) else 0
            team2_value = row[f'team2_{stat}'] if pd.notna(row[f'team2_{stat}']) else 0

            cumulative[stat][team1] += team1_value
            cumulative[stat][team2] += team2_value

            # Unified cumulative (including current stats)
            df.at[index, f'Cum {stat}'] = cumulative[stat][team1] + cumulative[stat][team2]

        # Increment games played for both teams AFTER calculation
        games_played[team1] += 1
        games_played[team2] += 1

    return df


In [1487]:
# Call the function to calculate stats
df_with_stats = calculate_weighted_stats_xp(data_df)

# Define the columns to display for checking
columns_to_display = [
    'Match ID', 'Team 1', 'Team 2',
    'team1_np_xg', 'team2_np_xg', 'Avg team1_np_xg', 'Avg team2_np_xg',
    'team1_expected_points', 'team2_expected_points',  'Avg team1_expected_points','Avg team2_expected_points', 'team_1_xg', 'team_2_xg',
    'Avg xG Team 1', 'Avg xG Team 2',
    'Weighted Avg xG Team 1', 'Weighted Avg xG Team 2',  'xG', 'xG.1','Avg xG','Avg xG.1', 'Weighted Avg xG', 'Weighted Avg xG.1'
]

# Filter and display the necessary columns
processed_data = df_with_stats[columns_to_display]

# Display the filtered DataFrame
processed_data.head(30) # Adjust the number of rows to inspect more games


/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_50686/1574674132.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2.461391' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, f'Cum {stat}'] = cumulative[stat][team1] + cumulative[stat][team2]
/var/folders/dy/ysjjzh0121qb38fzyck4zbdh0000gn/T/ipykernel_50686/1574674132.py:58: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2.8510999999999997' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.at[index, f'Cum {stat}'] = cumulative[stat][team1] + cumulative[stat][team2]


,Match ID,Team 1,Team 2,team1_np_xg,team2_np_xg,Avg team1_np_xg,Avg team2_np_xg,team1_expected_points,team2_expected_points,Avg team1_expected_points,Avg team2_expected_points,team_1_xg,team_2_xg,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,xG,xG.1,Avg xG,Avg xG.1,Weighted Avg xG,Weighted Avg xG.1
0,497410,Manchester Utd,Fulham,2.042680,0.418711,0.000000,0.000000,2.5696,0.2815,0.00000,0.00000,2.042680,0.418711,0.000000,0.000000,0.000000,0.000000,2.4,0.4,0.00,0.00,0.000000,0.000000
1,497411,Ipswich Town,Liverpool,0.342601,3.929060,0.000000,0.000000,0.0258,2.9562,0.00000,0.00000,0.342601,3.929060,0.000000,0.000000,0.000000,0.000000,0.5,2.6,0.00,0.00,0.000000,0.000000
2,497412,Arsenal,Wolves,1.628300,0.575835,0.000000,0.000000,2.1894,0.5796,0.00000,0.00000,1.628300,0.575835,0.000000,0.000000,0.000000,0.000000,1.2,0.5,0.00,0.00,0.000000,0.000000
3,497413,Everton,Brighton,0.405325,1.790830,0.000000,0.000000,0.3078,2.5287,0.00000,0.00000,0.405325,1.790830,0.000000,0.000000,0.000000,0.000000,0.5,1.4,0.00,0.00,0.000000,0.000000
4,497414,Newcastle Utd,Southampton,0.433489,1.954830,0.000000,0.000000,0.2774,2.5586,0.00000,0.00000,0.433489,1.954830,0.000000,0.000000,0.000000,0.000000,0.3,1.8,0.00,0.00,0.000000,0.000000
5,497415,Nottingham Forest,Bournemouth,1.244050,1.909150,0.000000,0.000000,0.8519,1.8944,0.00000,0.00000,1.244050,1.909150,0.000000,0.000000,0.000000,0.000000,1.3,1.2,0.00,0.00,0.000000,0.000000
6,497416,West Ham,Aston Villa,2.043530,2.670300,0.000000,0.000000,1.4690,1.2989,0.00000,0.00000,2.804700,2.670300,0.000000,0.000000,0.000000,0.000000,2.3,2.0,0.00,0.00,0.000000,0.000000
7,497417,Brentford,Crystal Palace,1.380640,1.147570,0.000000,0.000000,1.5885,1.0974,0.00000,0.00000,1.380640,1.147570,0.000000,0.000000,0.000000,0.000000,1.6,1.2,0.00,0.00,0.000000,0.000000
8,497418,Chelsea,Manchester City,1.061230,1.180380,0.000000,0.000000,1.2353,1.4402,0.00000,0.00000,1.061230,1.180380,0.000000,0.000000,0.000000,0.000000,1.0,0.8,0.00,0.00,0.000000,0.000000
9,497419,Leicester City,Tottenham,1.026140,1.407750,0.000000,0.000000,1.0574,1.6475,0.00000,0.00000,1.026140,1.407750,0.000000,0.000000,0.000000,0.000000,1.0,1.2,0.00,0.00,0.000000,0.000000


### Save the data

In [1398]:
processed_data.to_csv("updated_data.csv")

In [1399]:
updated_data = processed_data
updated_data.tail()

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Team_1_Wins_Last_4,H2H_Team_2_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,Unnamed: 0.1,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Unnamed: 0,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2
154,497562,Brighton,Crystal Palace,NaN,NaN,NaN,15,6,6,3,25,22,3,24,1.6,15,2,7,6,14,20,-6,13,1.2,2024-12-15,14:00,16,Sun,NaN,NaN,NaN,The American Express Stadium,NaN,11.0,50.8618,-0.0833,2024-12-15T14:00:00,NaN,0,0,0,2024-12-15_Brighton_Crystal Palace,CRY,BRI,NaN,NaN,False,False,https://understat.com/match/26756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,2,3,2024-12-15_Brighton_Crystal Palace,NaN,NaN,NaN,154.0,NaN,NaN,NaN,NaN,NaN,155.0,51,48,3.1875,3.0000,155.532181,210.166351,10.368812,14.011090,104.0,90.0,6.933333,6.000000,33.871453,42.033270,22.648889,18.000000,0,1.460000,4.65375,0,1.306667,3.920000,24.654340,22.474651,1.643623,1.498310,5.239047,4.494930
155,497565,Manchester City,Manchester Utd,NaN,NaN,NaN,15,8,3,4,27,21,6,27,0.8,15,5,4,6,19,18,1,19,1.4,2024-12-15,16:30,16,Sun,NaN,NaN,NaN,Etihad Stadium,NaN,14.0,53.4831,-2.2003,2024-12-15T16:30:00,NaN,0,0,0,2024-12-15_Manchester City_Manchester Utd,MUN,MCI,NaN,NaN,False,False,https://understat.com/match/26753,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,3,5,2024-12-15_Manchester City_Manchester Utd,NaN,NaN,NaN,155.0,NaN,NaN,NaN,NaN,NaN,156.0,48,49,3.0000,3.0625,167.388097,172.377026,11.159206,11.491802,207.0,105.0,13.800000,7.000000,33.477619,33.709285,41.400000,20.533333,0,1.913333,5.74000,0,1.406667,4.307917,31.567085,23.801301,2.104472,1.586753,6.313417,4.859432
156,497563,Chelsea,Brentford,NaN,NaN,NaN,15,9,4,2,35,18,17,31,2.6,15,7,2,6,31,28,3,23,2.0,2024-12-15,19:00,16,Sun,NaN,NaN,NaN,Stamford Bridge,NaN,9.0,51.4817,-0.1911,2024-12-15T19:00:00,NaN,0,0,0,2024-12-15_Chelsea_Brentford,BRE,CHE,NaN,NaN,False,False,https://understat.com/match/26757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,2,4,2024-12-15_Chelsea_Brentford,NaN,NaN,NaN,156.0,NaN,9.2,1.36,6.4,NaN,157.0,48,47,3.0000,2.9375,172.707411,193.965937,11.513827,12.931062,135.0,92.0,9.000000,6.133333,35.309071,37.069046,27.600000,17.582222,0,2.133333,6.40000,0,1.593333,4.680417,33.914606,25.884640,2.260974,1.725643,6.782921,5.069075
157,497568,Southampton,Tottenham,NaN,NaN,NaN,15,1,2,12,11,31,-20,5,0.2,15,6,2,7,31,19,12,20,0.8,2024-12-15,19:00,16,Sun,NaN,NaN,NaN,St Mary's Stadium,NaN,15.0,50.9058,-1.3913,2024-12-15T19:00:00,NaN,0,0,0,2024-12-15_Southampton_Tottenham,TOT,SOU,NaN,NaN,False,False,https://understat.com/match/26759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,4,2,2024-12-15_Southampton_Tottenham,NaN,NaN,NaN,157.0,NaN,NaN,NaN,NaN,NaN,158.0,54,46,3.37

### FE Average win/draw/points/goalscored ... stats

In [1401]:
import pandas as pd

def calculate_cumulative_stats(dataframe):
    # Initialize dictionaries to track cumulative stats and games played for each team
    cumulative_stats = {}
    games_played = {}

    # List of stats to track
    stats_columns = [
        'Won', 'Drawn', 'Lost', 'Goals Scored', 'Goals Conceded', 
        'Goal Difference', 'Points'
    ]

    # Initialize columns in the DataFrame to store cumulative and average stats for each team
    for stat in stats_columns:
        dataframe[f'Cum Team 1 {stat}'] = 0
        dataframe[f'Cum Team 2 {stat}'] = 0
        dataframe[f'Avg Team 1 {stat}'] = 0.0
        dataframe[f'Avg Team 2 {stat}'] = 0.0

    # Lists to store the values temporarily
    temp_cum_team1 = {stat: [] for stat in stats_columns}
    temp_cum_team2 = {stat: [] for stat in stats_columns}
    temp_avg_team1 = {stat: [] for stat in stats_columns}
    temp_avg_team2 = {stat: [] for stat in stats_columns}

    # Iterate over rows
    for index, row in dataframe.iterrows():
        team1 = row['Team 1']
        team2 = row['Team 2']

        if team1 not in cumulative_stats:
            cumulative_stats[team1] = {stat: 0 for stat in stats_columns}
            games_played[team1] = 0
        if team2 not in cumulative_stats:
            cumulative_stats[team2] = {stat: 0 for stat in stats_columns}
            games_played[team2] = 0

        for stat in stats_columns:
            # Update cumulative stats
            cumulative_stats[team1][stat] += row[f'Prev Team 1 {stat}']
            cumulative_stats[team2][stat] += row[f'Prev Team 2 {stat}']

        # Calculate averages using the count of games played so far
        for stat in stats_columns:
            if games_played[team1] > 0:
                avg_team_1_stat = cumulative_stats[team1][stat] / games_played[team1]
                temp_avg_team1[stat].append(avg_team_1_stat)
                temp_cum_team1[stat].append(cumulative_stats[team1][stat])
            else:
                temp_avg_team1[stat].append(0)
                temp_cum_team1[stat].append(0)
            
            if games_played[team2] > 0:
                avg_team_2_stat = cumulative_stats[team2][stat] / games_played[team2]
                temp_avg_team2[stat].append(avg_team_2_stat)
                temp_cum_team2[stat].append(cumulative_stats[team2][stat])
            else:
                temp_avg_team2[stat].append(0)
                temp_cum_team2[stat].append(0)

        # Increment games played after calculation
        games_played[team1] += 1
        games_played[team2] += 1

    # Assign lists to DataFrame columns
    for stat in stats_columns:
        dataframe[f'Cum Team 1 {stat}'] = temp_cum_team1[stat]
        dataframe[f'Cum Team 2 {stat}'] = temp_cum_team2[stat]
        dataframe[f'Avg Team 1 {stat}'] = temp_avg_team1[stat]
        dataframe[f'Avg Team 2 {stat}'] = temp_avg_team2[stat]

    return dataframe

# Assuming 'data' is your DataFrame loaded from 'your_data.csv'
updated_data = calculate_cumulative_stats(updated_data)

# Display selected rows with cumulative values and averages
columns_to_display = ['Match ID', 'Team 1', 'Team 2'] + [col for col in data.columns if 'Avg' in col or 'Cum' in col]
updated_data[columns_to_display].head()


,Match ID,Team 1,Team 2,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR
0,497410,Manchester Utd,Fulham,2,3,2.0,3.0
1,497411,Ipswich Town,Liverpool,5,2,5.0,2.0
2,497412,Arsenal,Wolves,2,5,2.0,5.0
3,497413,Everton,Brighton,2,2,2.0,2.0
4,497414,Newcastle Utd,Southampton,2,4,2.0,4.0


In [1402]:
# Displaying rows 20 to 30 from the DataFrame
# updated_data.iloc[60:100]

In [1403]:
updated_data.tail(12)

,Match ID,Team 1,Team 2,Team 1 Score,Team 2 Score,Winner,Prev Team 1 Played,Prev Team 1 Won,Prev Team 1 Drawn,Prev Team 1 Lost,Prev Team 1 Goals Scored,Prev Team 1 Goals Conceded,Prev Team 1 Goal Difference,Prev Team 1 Points,Prev Team 1 Form,Prev Team 2 Played,Prev Team 2 Won,Prev Team 2 Drawn,Prev Team 2 Lost,Prev Team 2 Goals Scored,Prev Team 2 Goals Conceded,Prev Team 2 Goal Difference,Prev Team 2 Points,Prev Team 2 Form,Date,Time,Wk,Day,xG,xG.1,Attendance,Venue,Referee,StadiumID,Latitude,Longitude,Datetime,Temperature,H2H_Team_1_Wins_Last_4,H2H_Team_2_Wins_Last_4,H2H_Draws_Last_4,Composite Key,team_2_code,team_1_code,team_1_xg,team_2_xg,is_result,has_data,url,team2_points,team2_expected_points,team2_np_xg,team2_np_xg_difference,team2_ppda,team2_deep_completions,team1_points,team1_expected_points,team1_np_xg,team1_np_xg_difference,team1_ppda,team1_deep_completions,gameweek,team_1_fdr,team_2_fdr,composite_key,B365H,B365D,B365A,Unnamed: 0.1,Score,Away Win Odds,Home Win Odds,Draw Odds,Precipitation,Unnamed: 0,Cum Team 1 FDR,Cum Team 2 FDR,Avg Team 1 FDR,Avg Team 2 FDR,Cum PPDA Team 1,Cum PPDA Team 2,Avg PPDA Team 1,Avg PPDA Team 2,Cum Deep Completions Team 1,Cum Deep Completions Team 2,Avg Deep Completions Team 1,Avg Deep Completions Team 2,Weighted Avg PPDA Team 1,Weighted Avg PPDA Team 2,Weighted Avg Deep Completions Team 1,Weighted Avg Deep Completions Team 2,Cum xG,Avg xG,Weighted Avg xG,Cum xG.1,Avg xG.1,Weighted Avg xG.1,Cum xG Team 1,Cum xG Team 2,Avg xG Team 1,Avg xG Team 2,Weighted Avg xG Team 1,Weighted Avg xG Team 2,Cum Team 1 Won,Cum Team 2 Won,Avg Team 1 Won,Avg Team 2 Won,Cum Team 1 Drawn,Cum Team 2 Drawn,Avg Team 1 Drawn,Avg Team 2 Drawn,Cum Team 1 Lost,Cum Team 2 Lost,Avg Team 1 Lost,Avg Team 2 Lost,Cum Team 1 Goals Scored,Cum Team 2 Goals Scored,Avg Team 1 Goals Scored,Avg Team 2 Goals Scored,Cum Team 1 Goals Conceded,Cum Team 2 Goals Conceded,Avg Team 1 Goals Conceded,Avg Team 2 Goals Conceded,Cum Team 1 Goal Difference,Cum Team 2 Goal Difference,Avg Team 1 Goal Difference,Avg Team 2 Goal Difference,Cum Team 1 Points,Cum Team 2 Points,Avg Team 1 Points,Avg Team 2 Points
147,497558,Tottenham,Chelsea,3.0,4.0,AWAY_TEAM,14,6,2,6,28,15,13,20,1.4,14,8,4,2,31,15,16,28,2.2,2024-12-08,16:30,15,Sun,2.8,2.8,61184.0,Tottenham Hotspur Stadium,Anthony Taylor,16.0,51.6047,-0.0664,2024-12-08T16:30:00,15.206185,0,0,0,2024-12-08_Tottenham_Chelsea,CHE,TOT,3.25488,2.57326,True,True,https://understat.com/match/26745,3.0,0.9833,1.05092,-2.20396,7.958333,16.0,0.0,1.8101,3.25488,2.20396,9.857143,8.0,15.0,4,4,2024-12-08_Tottenham_Chelsea,3.4,3.9,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,44,46,2.933333,3.066667,130.267048,164.749077,9.304789,11.767791,156.0,119.0,11.142857,8.500000,26.585112,35.303374,31.836735,25.500000,0,1.814286,5.321905,0,2.085714,6.396190,28.087009,31.341346,2.006215,2.238668,5.884897,6.865247,47,55,3.357143,3.928571,16,29,1.142857,2.071429,42,21,3.000000,1.500000,217,220,15.500000,15.714286,112,125,8.000000,8.928571,105,95,7.500000,6.785714,157,194,11.214286,13.857143
148,497559,West Ham,Wolves,2.0,1.0,HOME_TEAM,14,4,3,7,18,27,-9,15,0.8,14,2,3,9,22,36,-14,9,1.4,2024-12-09,20:00,15,Mon,1.0,1.4,62474.0,London Stadium,John Brooks,7.0,51.5386,-0.0164,2024-12-09T20:00:00,15.206185,0,0,0,2024-12-09_West Ham_Wolves,WOL,WHU,1.19808,1.47567,True,True,https://understat.com/match/26746,0.0,1.5825,1.47567,0.27759,8.037037,6.0,3.0,1.1481,1.19808,-0.27759,9.043478,7.0,15.0,2,2,2024-12-09_West Ham_Wolves,1.9,3.9,3.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46,47,3.066667,3.133333,198.047960,180.461828,14.146283,12.890131,94.0,72.0,6.714286,5.142857,44.459746,41.432563,21.102041,16.530612,0,1.471429,4.512381,0,0.992857,3.110952,22.163048,14.854259,1.583075,1.061018,4.854763,3.324525,30,7,2.142857,0.500000,24,23,1.714286,1.642857,51,75,3.642857,5.357143,134,145,9.571429,10.357143,184,279,13.142857,19.928571,-50,-134,-3.571429,-9.571429,114,44,8.142857,3.142857
149,497561,Arsenal,Everton,NaN,NaN,NaN,15,8,5,2,29,15,14,29,2.2,14,3,5,6,14,21,-7,14,1.0,

In [1404]:
updated_data.to_csv("fact_table.csv")